<a href="https://colab.research.google.com/github/ashish3127/ResnetDog_breed/blob/main/ResNet_Dog_breed_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
import tensorflow as tf
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [2]:
# drive mounting 
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Navigate to my drive directory to store the dataset
%cd /content/drive/MyDrive/Dog_breed_prediction

/content/drive/MyDrive/Dog_breed_prediction


In [4]:
train = '/content/drive/MyDrive/Dog_breed_prediction/train'
test ='/content/drive/MyDrive/Dog_breed_prediction/test'

In [5]:
def append_ext(fn):
    return fn+".jpg"
labels = pd.read_csv('/content/drive/MyDrive/Dog_breed_prediction/labels.csv',dtype=str)
sample_sub = pd.read_csv('/content/drive/MyDrive/Dog_breed_prediction/sample_submission.csv',dtype=str)
labels["id"] = labels["id"].apply(append_ext)
sample_sub["id"] = sample_sub["id"].apply(append_ext)

In [6]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [7]:
labels = labels.loc[labels['breed'].isin(['beagle','chihuahua','doberman','french_bulldog','golden_retriever','malamute','pug','saint_bernard','scottish_deerhound','tibetan_mastiff'])]

In [8]:
labels.shape

(841, 2)

In [9]:
sample_sub=sample_sub[['id','beagle','chihuahua','doberman','french_bulldog','golden_retriever','malamute','pug','saint_bernard','scottish_deerhound','tibetan_mastiff']]

In [10]:
#image_preprocessing
train_datagen=ImageDataGenerator( rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest' 
                                 )

In [11]:
train_generator=train_datagen.flow_from_dataframe(dataframe=labels,
                                                  directory=train,
                                                  x_col="id",
                                                  y_col="breed",
                                                  subset="training",
                                                  batch_size=32,
                                                  seed=42,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(224,224),
                                                  color_mode="rgb" 
                                                  )

Found 841 validated image filenames belonging to 10 classes.


In [12]:
valid_generator=train_datagen.flow_from_dataframe(dataframe=labels,
                                                  directory=train,
                                                  x_col="id",
                                                  y_col="breed",
                                                  subset="validation",
                                                  batch_size=32,
                                                  seed=42,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(224,224),
                                                  color_mode="rgb")

Found 0 validated image filenames belonging to 10 classes.


In [13]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=sample_sub,
                                                directory=test,
                                                x_col="id",
                                                y_col=None,
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(224,224),
                                                color_mode="rgb")

Found 10357 validated image filenames.


In [14]:
x,y = next(train_generator)

In [15]:
print(type(x))
print(x.shape)
print(y.shape)

<class 'numpy.ndarray'>
(32, 224, 224, 3)
(32, 10)


In [16]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [17]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [18]:
def ResNet50(input_shape=( 224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [19]:
base_model = ResNet50(input_shape=(224, 224, 3))


In [20]:
base_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [21]:
pretrained_model = tf.keras.applications.ResNet50V2(
        weights='imagenet',
        include_top=False ,
        input_shape=[224,224,3]
    )
pretrained_model.trainable = False
    
model = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Dense(256, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [22]:
opt=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer = opt ,
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [23]:
from tensorflow.python.keras.optimizer_v2.adam import Adam
base_model.compile(optimizer=Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [24]:
#early = tf.keras.callbacks.EarlyStopping( patience=2,
 #                                         min_delta=0.001,
  #                                        restore_best_weights=True)

In [25]:
#STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
#STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
#history = model.fit(train_generator,
#                    steps_per_epoch=STEP_SIZE_TRAIN,
#                    validation_data=valid_generator,
#                    validation_steps=STEP_SIZE_VALID,
#                    epochs=10,
#                    callbacks=[early],)

In [26]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/Dog_breed_prediction/model.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=4, min_lr=1e-5, mode='min')

In [27]:
history = model.fit(
                train_generator, 
                validation_data = valid_generator, 
                epochs = 10,
                callbacks=[checkpoint, lr_reducer]
                )


Epoch 1/10
27/27 [==============================] - 17s 460ms/step - loss: 2.6171 - accuracy: 0.1730
Epoch 2/10
27/27 [==============================] - 13s 469ms/step - loss: 1.7318 - accuracy: 0.4434
Epoch 3/10
27/27 [==============================] - 13s 461ms/step - loss: 1.2641 - accuracy: 0.5838
Epoch 4/10
27/27 [==============================] - 12s 459ms/step - loss: 1.0110 - accuracy: 0.7062
Epoch 5/10
27/27 [==============================] - 12s 460ms/step - loss: 0.9148 - accuracy: 0.7371
Epoch 6/10
27/27 [==============================] - 13s 463ms/step - loss: 0.7105 - accuracy: 0.7834
Epoch 7/10
27/27 [==============================] - 13s 468ms/step - loss: 0.7069 - accuracy: 0.7785
Epoch 8/10
27/27 [==============================] - 12s 460ms/step - loss: 0.5242 - accuracy: 0.8595
Epoch 9/10
27/27 [==============================] - 13s 461ms/step - loss: 0.5660 - accuracy: 0.8369
Epoch 10/10
27/27 [==============================] - 13s 461ms/step - loss: 0.5138 - accura